In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import torch
import pandas as pd
import glob
import os
import re
import numpy as np
import sys
from copy import deepcopy, copy
import pickle
sys.path.append('../../src')
from data import read_ATIS, load_pkl, load_glove_embed
from tensorly import cp_tensor, decomposition, tenalg
from rules.fsa_to_tensor import Automata
from rules.create_logic_mat_bias import create_mat_and_bias_with_empty_ATIS
from data import load_dict_ATIS, load_classification_dataset
from rules.tensor_func import tensor3_to_factors
from rules.fsa_to_tensor import dfa_to_tensor
from RE import PredictByRE1
from utils.utils import Args
import os
pd.set_option('display.max_rows', 300)

In [22]:
# ATIS
config_path = '../../model/11-6/1106090555-1604653555.7235456.res' 
model_path = '../../model/11-6/D0.9739-T0.9552-DI0.8655-TI0.8712-1106090555-1604653555.7040935-ATIS-0.model'
dfa_path = '../../data/ATIS/automata/automata.newrule.split.randomseed150.False.0.0003.0.pkl'
dataset = 'ATIS'

# # QC
config_path = '../../model/11-6/1106092247-1604654567.165155.res' 
model_path = '../../model/11-6/D0.8820-T0.8940-DI0.5020-TI0.5920-1106092247-1604654567.126257-TREC-0.model'
dfa_path = '../../data/TREC/automata/automata.split.randomseed.200.False.0.0021.0.pkl'
dataset = 'TREC'



_all = pickle.load(open(config_path,'rb'))
results_dict = _all['res']
args_dict = _all['args']
config = args_dict

model_parameters = torch.load(model_path, map_location='cpu')


automata_dicts = load_pkl(dfa_path)
dset = load_classification_dataset(dataset, '../../data/')
word2idx = dset['t2i']
automata = automata_dicts['automata']
V_embed, D1, D2 = automata_dicts['V'], automata_dicts['D1'], automata_dicts['D2']
language_tensor1, state2idx, wildcard_mat1, language1 = dfa_to_tensor(automata, word2idx)
complete_tensor = language_tensor1 + wildcard_mat1
print(V_embed.shape)
print(config['beta'])

D1_new = model_parameters['fsa_rnn.trans_r_1'].numpy()
D2_new = model_parameters['fsa_rnn.trans_r_2'].numpy()
V_embed_new = model_parameters['fsa_rnn.embed_r.weight'].numpy()[:-1] # V+1 R => V, R
word_embed_new = model_parameters['fsa_rnn.embedding.weight'].numpy()[:-1]# V+1 D => V, D
V_embed_generalized_new = model_parameters['fsa_rnn.embed_r_generalized'].numpy() # D x R
wildcard_mat_new = model_parameters['fsa_rnn.trans_wildcard'].numpy()
bias_new = model_parameters['bias'].numpy()
mat_new = model_parameters['mat'].numpy()

../../data/TREC/automata/automata.split.randomseed.200.False.0.0021.0.pkl
(8986, 200)
0.3


In [23]:
D2_new

array([[ 0.04113941,  0.00578136,  0.0214278 , ..., -0.08589418,
        -0.11406953, -0.01329235],
       [ 0.01657075, -0.0152354 ,  0.02324565, ..., -0.01573091,
        -0.01982415, -0.03651084],
       [-0.01272624, -0.02381838, -0.01476424, ..., -0.00373371,
         0.00577647, -0.04704226],
       ...,
       [-0.04664796,  0.05202688,  0.02019971, ..., -0.0008244 ,
         0.0081084 , -0.02956333],
       [-0.04258694,  0.04758633,  0.02350235, ..., -0.0096619 ,
         0.00265078, -0.02633813],
       [-0.00798957,  0.01635643, -0.01469606, ..., -0.04155448,
        -0.03224054,  0.02322396]], dtype=float32)

In [24]:
V_embed_generalized_recovered = np.dot(word_embed_new, V_embed_generalized_new)
V_embed_new_recovered = V_embed_generalized_recovered * (1 - config['beta']) + config['beta'] * V_embed_new
recovered_language_tensor = cp_tensor.cp_to_tensor([None,[V_embed_new_recovered, D1_new, D2_new]]) 
complete_recovered_tensor = recovered_language_tensor + wildcard_mat_new

In [25]:
new_rule_tensor = np.zeros_like(complete_recovered_tensor)
gamma = 0.3  # 0.7 for default ATIS
new_rule_tensor[complete_recovered_tensor > gamma] = 1
new_rule_tensor[complete_recovered_tensor < gamma] = 0
# new_rule_tensor = complete_recovered_tensor
print(new_rule_tensor.sum())
print(complete_tensor.sum())

685485.0
665722.0


In [7]:
params = {
    'complete_tensor': new_rule_tensor,
    'mat': mat_new,
    'bias': bias_new,
}

In [8]:
config['bz'] = 500
args = Args(config)

In [9]:
print('INIT DEV ACC: {}'.format(results_dict[int(args.seed)][0]))
print('INIT TEST ACC: {}'.format(results_dict[int(args.seed)][1]))

INIT DEV ACC: 0.502
INIT TEST ACC: 0.592


In [10]:
dset = load_classification_dataset(dataset, '../../data/')
print('Trained RE ACC')
_ , _,_,_,_,_ = PredictByRE1(args, params,  dset,)

Trained RE ACC
max_len: 39, avg_len: 12.213695871097684
max_len: 33, avg_len: 12.1
max_len: 19, avg_len: 9.534
(8987, 94, 94)
RE TRAIN ACC
total acc: 0.6201409869083585
0.6201409869083585
RE DEV ACC
total acc: 0.592
0.592
RE TEST ACC
total acc: 0.71
0.7100000000000001


In [11]:
args.seed

'0'

In [12]:
print(dset['t2i']['<pad>'])

8986


reset transitions based on the trained automata

In [13]:
import copy
t2i = dset['t2i']
i2t = dset['i2t']
automata_new = Automata()
automata_origin = Automata()
for k, v in automata.items():
    setattr(automata_new, k, v)
    setattr(automata_origin, k, v)
automata_new.transitions = dict()
n_vocab, n_state, _ = new_rule_tensor.shape
for s_from in range(n_state):
    for s_to in range(n_state):
        trans_info = new_rule_tensor[:,s_from, s_to]
        N = sum(trans_info) 
        if N >= len(t2i)-1:
            inp = '$'.format(N)
            automata_new.addtransition(s_from, s_to, inp)
        elif N > 15:
            inp = '${}'.format(N)
            automata_new.addtransition(s_from, s_to, inp)
        elif ((N > 0) and (N <= 15)):
            inp = set([i2t[i] for i in range(len(trans_info)) if trans_info[i]])
            automata_new.addtransition(s_from, s_to, inp)

In [14]:
automata_new.drawGraph(dataset)

In [15]:
# https://www.python.org/doc/essays/graphs/
def find_all_paths(transition_graph, start, end, path=[]):
    path = path + [start]
    if start == end:
        return [path]

    paths = []
    tos = transition_graph[start]
    
    nodes = [i for i in range(len(tos)) if tos[i] ]
    
    for node in nodes:
        
        if node not in path:
            newpaths = find_all_paths(transition_graph, node, end, path)
            for newpath in newpaths:
                paths.append(newpath)
    return paths

In [16]:
automata['finalstates_label']

{'ENTITY': [5, 7, 13],
 'DESCRIPTION': [17, 21, 23, 26, 30, 36, 37, 38, 39],
 'LOCATION': [42, 45, 50, 43],
 'HUMAN': [53, 56, 57, 65],
 'NUMERIC': [71],
 'ABBREVIATION': [88]}

In [17]:
def get_partial_automata(based_automata, label, new_rule_tensor):

    automata_partial = Automata()
    automata_partial.setstartstate(0)
    tensor_sum = new_rule_tensor.sum(0)
    transition_graph = np.zeros_like(tensor_sum)
    transition_graph[tensor_sum > 0.99] = 1

    all_paths = []
    goals = based_automata.finalstates_label[label]
    for goal in goals[0:3] :
        paths = find_all_paths(transition_graph, 0, goal)
        all_paths += paths
        automata_partial.addfinalstates(goal)
        for path in paths:
            for i in range(len(path)-1):
                try:
                    inps = based_automata.transitions[path[i]][path[i+1]]
                except:
                    continue
                automata_partial.addtransition(path[i], path[i+1], inps)        
                if path[i] in based_automata.transitions[path[i]]:
                    inps = based_automata.transitions[path[i]][path[i]]
                    automata_partial.addtransition(path[i], path[i], inps)
                if path[i+1] in based_automata.transitions[path[i+1]]:
                    inps = based_automata.transitions[path[i+1]][path[i+1]]
                    automata_partial.addtransition(path[i+1], path[i+1], inps)

    print(all_paths)
    return automata_partial

In [19]:
label = 'capacity' # for ATIS
automata_partial = get_partial_automata(automata_new, label, new_rule_tensor)

KeyError: 'capacity'

In [ ]:
automata_partial.drawGraph(label)